<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# 第二模块: 控制流
**上一节: [组合逻辑](2.2_comb_logic.ipynb)**<br>
**下一节: [时序逻辑](2.4_sequential_logic.ipynb)**

## 动机
到目前为止，在Chisel中软件和硬件之间有着很强的对应关系。
在控制流中，我们将在我们对两者的思考方式之间看到更大的分歧。
本模块介绍了生成器软件和硬件中的控制流。
如果重新连接到Chisel线会发生什么？
如何制作具有多个输入的多路复用器或连接器？
通过完成本模块，您可以找到这些问题以及更多问题的答案。

## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# 最后连接语义

<span style="color:blue">**示例: 重新赋值**</span><br>
正如之前所见，Chisel允许使用`:=`运算符连接组件。
出于各种原因，可以向同一组件发出多个连接语句。
当发生这种情况时，最后一条语句获胜。

In [ ]:
class LastConnect extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := 1.U
  io.out := 2.U
  io.out := 3.U
  io.out := 4.U
}

//  测试 LastConnect
test(new LastConnect) { c => c.io.out.expect(4.U) } // 断言输出正确为4
println("SUCCESS!!") // Scala 代码: 如果运行到这里，说明我们的测试通过了！

---
# `when`、`elsewhen`和`otherwise`
Chisel中条件逻辑的主要实现是`when`、`elsewhen`和`otherwise`结构。
这通常看起来像
```scala
when(someBooleanCondition) {
  // 当为真时要执行的操作
}.elsewhen(someOtherBooleanCondition) {
  // 在此条件下要执行的操作
}.otherwise {
  // 如果没有一个布尔条件为真时要执行的操作
}
```
它们必须按上述顺序出现，尽管后两者中的任何一个都可以省略。
可以有任意多个`elsewhen`子句。
任何为真的部分都终止该结构。
在`when`和其它结构的体中执行的操作可以是复杂的块，并且可能包含嵌套的`when`和其它结构。
**与** Scala 的 `if` 不同，`when` 关联的块不返回值。
不能这样说
```scala
val result = when(squareIt) { x * x }.otherwise { x }
```
这将**不起作用**。我们将在 *Wires* 部分讨论此问题的解决方案。

<span style="color:blue">**示例: Chisel 条件语句**</span><br>
下面是使用 `when` 结构的示例 `Module`。

In [ ]:
// Max3 返回其3个参数的最大值
class Max3 extends Module {
  val io = IO(new Bundle {
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out = Output(UInt(16.W))
  })
    
  when(io.in1 >= io.in2 && io.in1 >= io.in3) {
    io.out := io.in1  
  }.elsewhen(io.in2 >= io.in3) {
    io.out := io.in2 
  }.otherwise {
    io.out := io.in3
  }
}

// 测试 Max3
test(new Max3) { c =>
  // 验证三个输入的最大值是否正确
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(2.U)
  c.io.out.expect(6.U)  // 输入1应该是最大的
  c.io.in2.poke(7.U)
  c.io.out.expect(7.U)  // 现在输入2最大
  c.io.in3.poke(11.U)
  c.io.out.expect(11.U) // 现在输入3最大
  c.io.in3.poke(3.U)
  c.io.out.expect(7.U)  // 显示减小输入也起作用
  c.io.in1.poke(9.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(6.U)
  c.io.out.expect(9.U)  // 仍然可以得到最大值，即使是平局
}

println("SUCCESS!!") // Scala 代码: 如果运行到这里，说明我们的测试通过了！

---
# `Wire` 结构
让我们回到上面描述的 `when` 不返回值的限制。
Chisel `Wire` 结构是绕过这个问题的一种方式。
`Wire` 定义了一个电路组件，可以出现在连接 `:=` 运算符的右侧或左侧。

<span style="color:blue">**示例: 使用 Wires 的 4 输入排序**</span><br>
为了说明这一点，让我们制作一个小型组合排序器，将其四个数值输入排序为其四个数值输出。
为了使事情更清晰，考虑以下图表。数据沿着红色线在每一步中遵循左值小于右值的原则，沿着黑色线遵循左值大于右值的原则，进行值的交换。
![Sort4](images/Sorter4.png)
该图显示了一系列以 *row* 开头的单元格，我们将使用 `Wire` 构建这些单元格，其中特定复制或交换的结果被放置。 这段代码相当冗长，但您将看到以后缩小它的方法。

In [ ]:
/** Sort4将其4个输入排序为其4个输出 */
class Sort4 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(io.in0 < io.in1) {
    row10 := io.in0            // 保留前两个元素
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // 交换前两个元素
    row11 := io.in0
  }

  when(io.in2 < io.in3) {
    row12 := io.in2            // 保留最后两个元素
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // 交换最后两个元素
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(row11 < row12) {
    row21 := row11            // 保留中间2个元素
    row22 := row12
  }.otherwise {
    row21 := row12            // 交换中间两个元素
    row22 := row11
  }

  val row20 = Wire(UInt(16.W))
  val row23 = Wire(UInt(16.W))
  when(row10 < row13) {
    row20 := row10            // 保留中间2个元素
    row23 := row13
  }.otherwise {
    row20 := row13            // 交换中间两个元素
    row23 := row10
  }

  when(row20 < row21) {
    io.out0 := row20            // 保留前两个元素
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // 交换前两个元素
    io.out1 := row20
  }

  when(row22 < row23) {
    io.out2 := row22            // 保留前两个元素
    io.out3 := row23
  }.otherwise {
    io.out2 := row23            // 交换前两个元素
    io.out3 := row22
  }
}


// 这是测试程序
test(new Sort4) { c =>
  // 验证输入是否排序
  c.io.in0.poke(3.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(12.U)
  c.io.out0.expect(3.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(9.U)
  c.io.out3.expect(12.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(4.U)
  c.io.in2.poke(6.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)
}
println("SUCCESS!!") // Scala 代码: 如果运行到这里，说明我们的测试通过了！


这里有一个使用一些Scala `List` 特性的更详尽的测试程序。您将在以后的模块中看到更多 `List` 函数。

In [ ]:
// 这是测试程序
test(new Sort4) { c =>
  // 验证4个数字的所有可能排序是否都排序
  List(1, 2, 3, 4).permutations.foreach { case i0 :: i1 :: i2 :: i3 :: Nil =>
    println(s"Sorting $i0 $i1 $i2 $i3")
    c.io.in0.poke(i0.U)
    c.io.in1.poke(i1.U)
    c.io.in2.poke(i2.U)
    c.io.in3.poke(i3.U)
    c.io.out0.expect(1.U)
    c.io.out1.expect(2.U)
    c.io.out2.expect(3.U)
    c.io.out3.expect(4.U)
  }
}
println("SUCCESS!!") // Scala 代码: 如果运行到这里，说明我们的测试通过了！


---
# 练习

<span style="color:red">**练习: 多项式电路**</span><br>
构建一个将计算这些多项式值的 `Module`。
- $x^2 - 2x + 1$
- $2x^2 + 6x + 3$
- $4x^2 - 10x -5$

选择输入将确定要计算哪个多项式。使用 `Wire`，使得 $x^2$ 计算仅出现在模块中一次，并且与输出有单一连接。

首先，让我们使用测试驱动的开发，使用Scala编写一个模型。完成以下函数定义以通过下面的断言。这不是详尽的检查，而是健全性检查。

In [ ]:
def poly0(x: Int): Int = ???
def poly1(x: Int): Int = ???
def poly2(x: Int): Int = ???

assert(poly0(0) == 1)
assert(poly1(0) == 3)
assert(poly2(0) == -5)

assert(poly0(1) == 0)
assert(poly1(1) == 11)
assert(poly2(1) == -11)

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def poly0(x: Int): Int = x*x - 2*x + 1
def poly1(x: Int): Int = 2*x*x + 6*x + 3
def poly2(x: Int): Int = 4*x*x - 10*x - 5
</pre></article></div></section></div>

为了使其更容易，让我们创建一个函数，它的工作方式类似于我们期望的硬件模块。使用Scala `if` 语句根据 `select` 输入选择多项式。

In [ ]:
def poly(select: Int, x: Int): Int = {
  ???
}

assert(poly(1, 0) == 3)
assert(poly(1, 1) == 11)
assert(poly(2, 1) == -11)

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def poly(select: Int, x: Int): Int = {
  if(select == 0) {
    poly0(x)
  }
  else if(select == 1) {
    poly1(x)
  }
  else {
    poly2(x)
  }
}
</pre></article></div></section></div>

看起来值是正确的。现在使用以下模板来实现你的电路。

In [ ]:
// 计算多项式
class Polynomial extends Module {
  val io = IO(new Bundle {
    val select = Input(UInt(2.W))
    val x = Input(SInt(32.W))
    val fOfX = Output(SInt(32.W))
  })
    
  val result = Wire(SInt(32.W))  
  val square = Wire(SInt(32.W))  
  
  ???

  io.fOfX := result  
}

// 测试 Polynomial
test(new Polynomial) { c =>
  for(x <- 0 to 20) {
    for(select <- 0 to 2) {
      c.io.select.poke(select.U)
      c.io.x.poke(x.S)
      c.io.fOfX.expect(poly(select, x).S)
    }
  }
}
println("SUCCESS!!") // Scala 代码: 如果运行到这里，说明我们的测试通过了！

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  square := io.x * io.x
  when(io.select === 0.U) {
    result := (square - (2.S * io.x)) + 1.S
  }.elsewhen(io.select === 1.U) {
    result := (2.S * square) + (6.S * io.x) + 3.S
  }.otherwise {
    result := (4.S * square) - (10.S * io.x) - 5.S
  }
</pre></article></div></section></div>

<span style="color:red">**练习: 有限状态机**</span><br>
使用卡诺图来优化状态机的逻辑是繁琐的，而且由综合工具解决。它还会产生难以理解和阅读的代码。因此，我们将使用 Chisel 控制流和最后连接语义编写一个更明智的状态机。

研究生在他们的职业生涯中经历四个状态：空闲、编码、写作和毕业。这些状态基于三个输入进行转换：咖啡、他们想出的点子和顾问对他们取得进展的压力。一旦毕业，他们就会回到空闲状态。下面的FSM图显示了这些状态和转换。任何未标记的转换（即没有输入时）都会将研究生带回空闲状态而不是保留在当前状态。输入优先级为咖啡 > 点子 > 压力，因此当处于空闲状态并同时接收咖啡和压力时，研究生将转到编码状态。

<img src="images/fsm.png" width="500" />

首先，我们将构建一个模型来测试我们的硬件。完成我们状态机的以下功能描述。它有四个输入。输出是下一个状态。状态映射已经提供给你，你可以像 `states("grad")` 一样访问它。

In [ ]:
// 状态映射
def states = Map("idle" -> 0, "coding" -> 1, "writing" -> 2, "grad" -> 3)

// 生活充满了问号
def gradLife (state: Int, coffee: Boolean, idea: Boolean, pressure: Boolean): Int = {
  var nextState = states("idle")
  ???
  nextState
}

// 一些合理性检查
(0 until states.size).foreach{ state => assert(gradLife(state, false, false, false) == states("idle")) }
assert(gradLife(states("writing"), true, false, true) == states("writing"))
assert(gradLife(states("idle"), true, true, true) == states("coding"))
assert(gradLife(states("idle"), false, true, true) == states("idle"))
assert(gradLife(states("grad"), false, false, false) == states("idle"))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  if (state == states("idle")) {
    if      (coffee) { nextState = states("coding") }
    else if (idea) { nextState = states("idle") }
    else if (pressure) { nextState = states("writing") }
  } else if (state == states("coding")) {
    if      (coffee) { nextState = states("coding") } 
    else if (idea || pressure) { nextState = states("writing") }
  } else if (state == states("writing")) {
    if      (coffee || idea) { nextState = states("writing") }
    else if (pressure) { nextState = states("grad") }
  }
</pre></article></div></section></div>

由于您尚未学习时序逻辑，当前状态是 `Module` 的输入，下一个状态是输出，与之前的 `gradLife` 函数一样。现在在 Chisel 中实现状态机以通过测试器。Chisel 为我们提供了一个方便的状态机映射函数，称为 `Enum`。要使用这些状态，请将它们视为 `UInt` 字面值。请记住，硬件相等性使用三重等号符号执行！

In [ ]:
// 生活变得更难
class GradLife extends Module {
  val io = IO(new Bundle {
    val state = Input(UInt(2.W))
    val coffee = Input(Bool())
    val idea = Input(Bool())
    val pressure = Input(Bool())
    val nextState = Output(UInt(2.W))
  })
    
  val idle :: coding :: writing :: grad :: Nil = Enum(4)
  
  ???
}

// 测试
test(new GradLife) { c =>
  // 验证硬件是否与黄金模型匹配
  for (state <- 0 to 3) {
    for (coffee <- List(true, false)) {
      for (idea <- List(true, false)) {
        for (pressure <- List(true, false)) {
          c.io.state.poke(state.U)
          c.io.coffee.poke(coffee.B)
          c.io.idea.poke(idea.B)
          c.io.pressure.poke(pressure.B)
          c.io.nextState.expect(gradLife(state, coffee, idea, pressure).U)
        }
      }
    }
  }
}
println("SUCCESS!!") // Scala 代码: 如果运行到这里，说明我们的测试通过了！

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-5" />
<label for="check-5"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  io.nextState := idle
  when (io.state === idle) {
    when      (io.coffee) { io.nextState := coding } 
    .elsewhen (io.idea) { io.nextState := idle }
    .elsewhen (io.pressure) { io.nextState := writing }
  } .elsewhen (io.state === coding) {
    when      (io.coffee) { io.nextState := coding } 
    .elsewhen (io.idea || io.pressure) { io.nextState := writing }
  } .elsewhen (io.state === writing) {
    when      (io.coffee || io.idea) { io.nextState := writing }
    .elsewhen (io.pressure) { io.nextState := grad }
  }
</pre></article></div></section></div>

---
# 完成了！

[返回顶部。](#top)